In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
import os
import requests
from datetime import datetime
import time

In [2]:
dfs = {}

In [3]:
dfbc = pd.read_csv('bitcoin_historical_data.csv') #bring forth bc historical data

In [4]:
dfbc.head() #confirm loading

,Date,Price,Open,High,Low,Vol.,Change %
0,03/24/2024,"67,211.9","64,036.5","67,587.8","63,812.9",65.59K,4.96%
1,03/23/2024,"64,037.8","63,785.6","65,972.4","63,074.9",35.11K,0.40%
2,03/22/2024,"63,785.5","65,501.5","66,633.3","62,328.3",72.43K,-2.62%
3,03/21/2024,"65,503.8","67,860.0","68,161.7","64,616.1",75.26K,-3.46%
4,03/20/2024,"67,854.0","62,046.8","68,029.5","60,850.9",133.53K,9.35%


In [5]:
dfbc.drop_duplicates(inplace=True)  #clean?

In [6]:
#converting dfbc Date to datetime
dfbc['Date'] = pd.to_datetime(dfbc['Date'], format = '%m/%d/%Y')

In [7]:
dfbc.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2024-03-24,"67,211.9","64,036.5","67,587.8","63,812.9",65.59K,4.96%
1,2024-03-23,"64,037.8","63,785.6","65,972.4","63,074.9",35.11K,0.40%
2,2024-03-22,"63,785.5","65,501.5","66,633.3","62,328.3",72.43K,-2.62%
3,2024-03-21,"65,503.8","67,860.0","68,161.7","64,616.1",75.26K,-3.46%
4,2024-03-20,"67,854.0","62,046.8","68,029.5","60,850.9",133.53K,9.35%


In [8]:
directory = '.'
json_files = [f for f in os.listdir('.') if f.endswith('.json')]

for file in json_files:
    print(file)

av_transactions_per_block.json
exchange_trade_volume.json
circulating_bitcoin_total.json
estimated_transaction_volume_usd.json
fees_per-_transaction.json
market_value_to_realised_value.json
transactions_total.json
bitcoin_profitable_days.json


In [9]:
for json_file in json_files:
    try:
        df = pd.read_json(json_file)

        print(f"\nConverting {json_file}:")
        print(df.head())
        print(df.columns)

        csv_file = json_file.replace('.json','csv')
        df.to_csv(csv_file, index=False)
        print(f"Saved as {csv_file}")

    except Exception as e:
        print(f"Error with {json_file}: {e}")

Error with av_transactions_per_block.json: All arrays must be of the same length

Converting exchange_trade_volume.json:
        metric1       metric2                  trade-volume  \
0  trade-volume  market-price  {'x': 1230940800000, 'y': 0}   
1  trade-volume  market-price  {'x': 1231286400000, 'y': 0}   
2  trade-volume  market-price  {'x': 1231632000000, 'y': 0}   
3  trade-volume  market-price  {'x': 1231977600000, 'y': 0}   
4  trade-volume  market-price  {'x': 1232323200000, 'y': 0}   

                   market-price    type average timespan  
0  {'x': 1230940800000, 'y': 0}  linear      1d      all  
1  {'x': 1231286400000, 'y': 0}  linear      1d      all  
2  {'x': 1231632000000, 'y': 0}  linear      1d      all  
3  {'x': 1231977600000, 'y': 0}  linear      1d      all  
4  {'x': 1232323200000, 'y': 0}  linear      1d      all  
Index(['metric1', 'metric2', 'trade-volume', 'market-price', 'type', 'average',
       'timespan'],
      dtype='object')
Saved as exchange_trade_

In [10]:
csv_file = json_file.replace('.json','csv')
df.to_csv(csv_file, index=False)

In [11]:
with open('av_transactions_per_block.json', 'r') as f:
    data = json.load(f)
print(type(data))
print(data.keys())

df_tranblock = pd.DataFrame(data['n-transactions-per-block'])
df_tranblock['Date'] = pd.to_datetime(df_tranblock['x'],unit='ms')
df_tranblock['Transactions Per Block'] = df_tranblock['y'].astype(float)
df_tranblock=df_tranblock[['Date','Transactions Per Block']]
print(df_tranblock.head())
print(df_tranblock.shape)

df_tranblock.to_csv('av_transactions_per_block.csv', index=False)

<class 'dict'>
dict_keys(['metric1', 'metric2', 'n-transactions-per-block', 'market-price', 'type', 'average', 'timespan'])
        Date  Transactions Per Block
0 2009-01-17                1.000000
1 2009-01-21                1.019417
2 2009-01-25                1.005236
3 2009-01-29                1.008264
4 2009-02-02                1.007812
(1481, 2)


In [12]:
with open('estimated_transaction_volume_usd.json', 'r') as f:
    data = json.load(f)
print(type(data))
print(data.keys())

df_estvol = pd.DataFrame(data['estimated-transaction-volume-usd'])
df_estvol['Date'] = pd.to_datetime(df_estvol['x'],unit='ms')
df_estvol['Estimated Transaction Volume'] = df_estvol['y'].astype(float)
df_estvol=df_estvol[['Date','Estimated Transaction Volume']]
print(df_estvol.head())
print(df_estvol.shape)

df_estvol.to_csv('estimated_transaction_volume_usd.csv', index=False)

<class 'dict'>
dict_keys(['metric1', 'metric2', 'estimated-transaction-volume-usd', 'market-price', 'type', 'average', 'timespan'])
        Date  Estimated Transaction Volume
0 2010-08-28                      861.9296
1 2010-09-01                     1010.3004
2 2010-09-05                     1828.3764
3 2010-09-09                     1023.9252
4 2010-09-13                     1688.8110
(1329, 2)


In [13]:
with open('fees_per-_transaction.json', 'r') as f:
    data = json.load(f)
print(type(data))
print(data.keys())

df_fees = pd.DataFrame(data['fees-usd-per-transaction'])
df_fees['Date'] = pd.to_datetime(df_fees['x'],unit='ms')
df_fees['Fees Per Transaction'] = df_fees['y'].astype(float)
df_fees=df_fees[['Date','Fees Per Transaction']]
print(df_fees.head())
print(df_fees.shape)

df_fees.to_csv('fees_per_transaction.csv', index=False)

<class 'dict'>
dict_keys(['metric1', 'metric2', 'fees-usd-per-transaction', 'market-price', 'type', 'average', 'timespan'])
        Date  Fees Per Transaction
0 2009-01-17                   0.0
1 2009-01-21                   0.0
2 2009-01-25                   0.0
3 2009-01-29                   0.0
4 2009-02-02                   0.0
(1481, 2)


In [14]:
with open('market_value_to_realised_value.json', 'r') as f:
    data = json.load(f)
print(type(data))
print(data.keys())

df_mvrv = pd.DataFrame(data['mvrv'])
df_mvrv['Date'] = pd.to_datetime(df_mvrv['x'],unit='ms')
df_mvrv['Market Value to Realised Value'] = df_mvrv['y'].astype(float)
df_mvrv=df_mvrv[['Date','Market Value to Realised Value']]

print(df_mvrv.head())
print(df_mvrv.shape)


df_mvrv.to_csv('market_value_to_realised_value.csv', index=False)

<class 'dict'>
dict_keys(['metric1', 'metric2', 'mvrv', 'market-price', 'type', 'average', 'timespan'])
                 Date  Market Value to Realised Value
0 2010-10-09 20:41:19                        6.352667
1 2010-10-13 02:47:11                        6.353138
2 2010-10-15 22:54:12                        5.641057
3 2010-10-17 23:55:18                        5.150964
4 2010-10-20 03:06:59                        4.576734
(1500, 2)


In [15]:
with open('transactions_total.json', 'r') as f:
    data = json.load(f)
    print(type(data))
    print(data.keys())

df_tran = pd.DataFrame(data['n-transactions-total'])

df_tran['Date'] = pd.to_datetime(df_tran['x'],unit='ms')
df_tran['Confirmed Payments'] = df_tran['y'].astype(float)
df_tran = df_tran[['Date','Confirmed Payments']]

print(df_tran.head())
print(df_tran.shape)

#save as csv
df_tran.to_csv('transactions_total.csv', index=False)

<class 'dict'>
dict_keys(['metric1', 'metric2', 'n-transactions-total', 'market-price', 'type', 'average', 'timespan'])
        Date  Confirmed Payments
0 2009-01-03                 1.0
1 2009-01-12               170.0
2 2009-01-16               658.0
3 2009-01-20              1102.0
4 2009-01-24              1498.0
(1483, 2)


In [16]:
json_files = [
    'av_transactions_per_block.json',
    'circulating_bitcoin_total.json',
    'estimated_transaction_volume_usd.json',
    'fees_per-_transaction.json',
    'market_value_to_realised_value.json',
    'transactions_total.json'

]
key_map = {
    'av_transactions_per_block.json': 'n-transactions-per-block',
    'circulating_bitcoin_total.json':'total-bitcoins',
    'estimated_transaction_volume_usd.json': 'estimated-transaction-volume-usd',
    'fees_per-_transaction.json': 'fees-usd-per-transaction',
    'market_value_to_realised_value.json': 'mvrv',
    'transactions_total.json': 'n-transactions-total'

}
names = {
    'n-transactions-per-block' : 'Transactions Per Block',
    'total-bitcoins' : 'Circulating Bitcoin',
    'estimated-transaction-volume-usd' : 'Estimated Transaction Volume',
    'fees-usd-per-transaction' : 'Fees Per Transaction',
    'mvrv' : 'Market Value to Realised Value',
    'n-transactions-total' : 'Confirmed Payments'
}

for json_file in json_files:
    print(f"\nProcessing {json_file}...")
    with open(json_file, 'r') as f:
        data = json.load(f)
    key = key_map[json_file]
    
    if key not in data:
        print(f"Error: Key {key} not in {json_file}")
        continue
        
    df = pd.DataFrame(data[key])
    if 'y' not in df or df['y'].isna().all():
        print(f"Error: No valid data in {json_file}")
        continue
    
    # Convert to correct types, with explicit error handling
    df['Date'] = pd.to_datetime(df['x'], unit='ms', errors='coerce')
    df[names[key]] = pd.to_numeric(df['y'], errors='coerce')
        
    df = df[['Date', names[key]]]
    df = df.set_index('Date').resample('D').interpolate(method='linear').reset_index()
    dfs[json_file] = df
    
    print(f"{json_file}: {df['Date'].min()} to {df['Date'].max()}, {len(df)} rows")
    print(f"Column '{names[key]}' has {df[names[key]].isna().sum()} NaN values")

dfbc['Date'] = pd.to_datetime(dfbc['Date'])
def clean_numeric(value):
    if isinstance(value, str):
        value = value.replace(',', '')  # Remove commas
        if 'K' in value:
            value = float(value.replace('K', '')) * 1000
        return pd.to_numeric(value, errors='coerce')
    return value

dfbc['Price'] = dfbc['Price'].apply(clean_numeric)
dfbc['Vol.'] = dfbc['Vol.'].apply(clean_numeric)
dfbc['Change %'] = dfbc['Change %'].str.rstrip('%').astype(float)
dfbc_newset = dfbc[['Date', 'Price', 'Vol.', 'Change %']]
dfs['dfbc'] = dfbc_newset
print(f"dfbc: {dfbc_newset['Date'].min()} to {dfbc_newset['Date'].max()}, {len(dfbc_newset)} rows")


Processing av_transactions_per_block.json...
av_transactions_per_block.json: 2009-01-17 00:00:00 to 2025-04-03 00:00:00, 5921 rows
Column 'Transactions Per Block' has 0 NaN values

Processing circulating_bitcoin_total.json...
circulating_bitcoin_total.json: 2009-01-10 00:00:00 to 2025-04-03 00:00:00, 5928 rows
Column 'Circulating Bitcoin' has 5928 NaN values

Processing estimated_transaction_volume_usd.json...
estimated_transaction_volume_usd.json: 2010-08-28 00:00:00 to 2025-04-04 00:00:00, 5334 rows
Column 'Estimated Transaction Volume' has 0 NaN values

Processing fees_per-_transaction.json...
fees_per-_transaction.json: 2009-01-17 00:00:00 to 2025-04-03 00:00:00, 5921 rows
Column 'Fees Per Transaction' has 0 NaN values

Processing market_value_to_realised_value.json...
market_value_to_realised_value.json: 2010-10-09 00:00:00 to 2025-04-04 00:00:00, 5292 rows
Column 'Market Value to Realised Value' has 1755 NaN values

Processing transactions_total.json...
transactions_total.json: 

In [17]:
# Improved merge process with debugging
print("\nStarting merge process...")
dffin = dfs['av_transactions_per_block.json'].copy()
print(f"Initial dataframe columns: {dffin.columns.tolist()}")

for json_file in json_files[1:]:
    if json_file not in dfs:
        print(f"Warning: {json_file} not in dfs dictionary, skipping")
        continue
        
    right = dfs[json_file]
    print(f"Merging with {json_file}, columns: {right.columns.tolist()}")
    
    # Debug the dataframe before merging
    print(f"Right dataframe has {right[right.columns[1]].isna().sum()} NaN values out of {len(right)} rows")
    
    # Use outer join to preserve all data
    dffin = pd.merge(dffin, right, on='Date', how='inner')
    print(f"After merging {json_file}, shape: {dffin.shape}, columns: {dffin.columns.tolist()}")

# Merge with bitcoin price data
dffin = pd.merge(dffin, dfs['dfbc'], on='Date', how='inner')

# Sort by date and handle missing values
dffin = dffin.sort_values('Date')
print(f"Before ffill/bfill, NaN counts:\n{dffin.isna().sum()}")

dffin = dffin.ffill().bfill()

# Apply date filters if needed
dffin = dffin[dffin['Date'] >= '2010-10-09']
dffin = dffin[dffin['Date'] <= '2024-03-24']

# Final column check and selection
all_columns = dffin.columns.tolist()
print(f"All available columns: {all_columns}")

columns = ['Date', 'Transactions Per Block', 
           'Estimated Transaction Volume', 'Fees Per Transaction', 
           'Market Value to Realised Value', 'Confirmed Payments', 
           'Price', 'Vol.', 'Change %']

# Check if all expected columns exist
for col in columns:
    if col not in all_columns:
        print(f"WARNING: Column '{col}' not found in merged dataframe!")

dffin = dffin[columns]
dffin['Vol.'] = dffin['Vol.'].fillna(0)


Starting merge process...
Initial dataframe columns: ['Date', 'Transactions Per Block']
Merging with circulating_bitcoin_total.json, columns: ['Date', 'Circulating Bitcoin']
Right dataframe has 5928 NaN values out of 5928 rows
After merging circulating_bitcoin_total.json, shape: (5921, 3), columns: ['Date', 'Transactions Per Block', 'Circulating Bitcoin']
Merging with estimated_transaction_volume_usd.json, columns: ['Date', 'Estimated Transaction Volume']
Right dataframe has 0 NaN values out of 5334 rows
After merging estimated_transaction_volume_usd.json, shape: (5333, 4), columns: ['Date', 'Transactions Per Block', 'Circulating Bitcoin', 'Estimated Transaction Volume']
Merging with fees_per-_transaction.json, columns: ['Date', 'Fees Per Transaction']
Right dataframe has 0 NaN values out of 5921 rows
After merging fees_per-_transaction.json, shape: (5333, 5), columns: ['Date', 'Transactions Per Block', 'Circulating Bitcoin', 'Estimated Transaction Volume', 'Fees Per Transaction']
Mer

In [18]:
print(dffin.head())
print(dffin.tail())


        Date  Transactions Per Block  Estimated Transaction Volume  \
0 2010-10-09                2.137972                   4297.880700   
1 2010-10-10                2.144458                   3621.558750   
2 2010-10-11                2.150943                   2945.236800   
3 2010-10-12                2.076484                   3295.342325   
4 2010-10-13                2.002025                   3645.447850   

   Fees Per Transaction  Market Value to Realised Value  Confirmed Payments  \
0          1.901163e-10                        1.028991            126453.0   
1          2.851744e-10                        1.028991            126831.0   
2          3.802326e-10                        1.028991            127185.5   
3          2.851744e-10                        1.028991            127540.0   
4          1.901163e-10                        1.028991            127894.5   

   Price      Vol.  Change %  
0    0.1  187850.0       0.0  
1    0.1   50680.0       0.0  
2    0.1   

In [20]:
# Normalize
features = dffin.drop('Date', axis=1)
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
scaled_df = pd.DataFrame(scaled_features, columns=features.columns)
scaled_df['Date'] = dffin['Date'].values
print(scaled_df.head())
print(scaled_df.tail())
print(scaled_df.shape)
scaled_df.to_csv('dffin_scaled.csv', index=False)

   Transactions Per Block  Estimated Transaction Volume  Fees Per Transaction  \
0                0.000102                  1.104717e-07          3.176098e-12   
1                0.000104                  7.596327e-08          4.764147e-12   
2                0.000105                  4.145487e-08          6.352196e-12   
3                0.000088                  5.931853e-08          4.764147e-12   
4                0.000071                  7.718218e-08          3.176098e-12   

   Market Value to Realised Value  Confirmed Payments  Price      Vol.  \
0                             0.0        0.000000e+00    0.0  0.187800   
1                             0.0        3.858333e-07    0.0  0.050542   
2                             0.0        7.476796e-07    0.0  0.013929   
3                             0.0        1.109526e-06    0.0  0.025506   
4                             0.0        1.471372e-06    0.0  0.047420   

   Change %       Date  
0  0.145185 2010-10-09  
1  0.145185 2010-1